In [1]:
1+2



3

In [2]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl
import h5py
import math

from util import *

import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers
from unc_funcs import *

/icarus/app/users/jdyer/dev_sbnana_v09_69_01/srcs/sbnana/sbnana/SBNAna/icarus-analysis-villiage/pyana/dimuon-tools/env/lib/python3.9/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Define Sidebands

Possible Cuts to Flip:

- NuMI Angle
- Opening Angle
- Track length
    
Considerations:

- When checking distributions, I should use the benchmark models that we have the greatest sensitivity to. This means I will need to rescale the weights to the optimal theta or cl and fa. 
- In the sidebands, the (optimal) signal benchmarks should have fewer than 10 POT-normalized events. Ideally there will be a lot of coherent pion events there. 
    - Follow up after discussion with Gray 10/23: it is okay if there are more than 10 signal events in the sideband as long as there are a lot more signal events in the signal region. But you would need to be careful about how you are modeling your background in the side region then. 
    - Note that if the ALPs we are most sensitive to exist, we probably would have already noticed them in the detector.


In [3]:
savestuff = False

POTSTR = "2e20 POT"

dir = '/icarus/data/users/jdyer/muon_selection/dfs/all_nus_bg/with_kin/'
evtdf = pd.read_pickle(dir + 'final_df')

#categories = make_categories(evtdf)

# HPS masses to consider: 220, 240, 260  
higgs_files = [
    "/icarus/data/users/gputnam/dimuon-data/higgs-gen-A/higgs_M220_DMCP2023G_evt_new.df",
    "/icarus/data/users/gputnam/dimuon-data/higgs-gen-A/higgs_M240_DMCP2023G_evt_new.df",
    "/icarus/data/users/gputnam/dimuon-data/higgs-gen-A/higgs_M260_DMCP2023G_evt_new.df",
]
higgs_mcdfs = [pd.read_hdf(f, key="mch") for f in higgs_files]
higgs_masses = [int(round(df.iloc[(0)].M*1000.)) for df in higgs_mcdfs]
old_thetas = [float(df.iloc[(0)].C1) for df in higgs_mcdfs]
best_thetas = [9.5e-4, 7e-4, 1e-4]

# ALP masses to consider: 300, 350, 400, 450
alp_nosup_files = [
    "/icarus/data/users/gputnam/dimuon-data/higgs-gen-A/axion-no-suppression/alp_M300_DMC2023G_evt.df",
    "/icarus/data/users/gputnam/dimuon-data/higgs-gen-A/axion-no-suppression/alp_M350_DMC2023G_evt.df",
    "/icarus/data/users/gputnam/dimuon-data/higgs-gen-A/axion-no-suppression/alp_M400_DMC2023G_evt.df",
    "/icarus/data/users/gputnam/dimuon-data/higgs-gen-A/axion-no-suppression/alp_M450_DMC2023G_evt.df",
]
alp_nosup_mcdfs = [pd.read_hdf(f, key="mch") for f in alp_nosup_files]
alp_nosup_masses = [int(round(df.iloc[(0)].M*1000.)) for df in alp_nosup_mcdfs]
old_cls = [float(df.iloc[(0)].C2) for df in alp_nosup_mcdfs]
old_fas = [float(df.iloc[(0)].C1) for df in alp_nosup_mcdfs]
best_fas = [9e-4, 9e-4, 9e-4, 7e-5]
best_cls = [1/1000., 1/1000., 1/1000., 1/300.]

# CATEGORIES

categories_to_start = make_categories(evtdf)
categories = categories_to_start[:3] + categories_to_start[5:9] + categories_to_start[12:]
for sample in categories:
    print(sample.name)


$M_S$ = 220, $\theta_S$ = 5e-05
$M_S$ = 240, $\theta_S$ = 5e-05
$M_S$ = 260, $\theta_S$ = 5e-05
$M_{ALP}$ = 300, $c$ = 0.01, $fa$ = 100000.0
$M_{ALP}$ = 350, $c$ = 0.01, $fa$ = 100000.0
$M_{ALP}$ = 400, $c$ = 0.01, $fa$ = 100000.0
$M_{ALP}$ = 450, $c$ = 0.01, $fa$ = 100000.0
$\nu$
Cosmic


In [18]:
evtdf


slc                          \
                                     is_clear_cosmic      vertex               
                                                               x           y   
                                                                               
                                                                               
                                                                               
sample __ntuple entry rec.slc..index                                           
0      10       6     0                            0 -137.550140   80.211090   
       49       2     1                            0 -139.672394  -60.575729   
       87       20    10                           0  210.485367  -56.962399   
       105      15    14                           0  142.216400  -46.298374   
       110      13    0                            0 -165.103683 -161.912247   
..                                               ...         ...         ...   
12     4279     172   1                            0 -187.539886   23.401184   
       4444     45    0                            0 -198.929413 -101.789307   
       4502     127   19                           0  255.800964  -55.191528   
       6150     236   17                           0  214.587585   21.150330   
       6645     120   0                            0 -160.763794   59.780487   

                                                                               \
                                                 self    tmatch                 
                                               z            eff       pur idx   
                                                                                
                                                                                
                                                                                
sample __ntuple entry rec.slc..index                                            
0      10       6     0               556.435852  182  0.922363  0.988839   0   
       49       2     1              -293.824646  250  0.709152  0.944103   0   
       87       20    10             -195.739227  130  0.867942  0.989385   0   
       105      15    14              404.727722  176  0.848213  0.992334   0   
       110      13    0               615.129761   72  0.893664  0.994344   0   
..                                           ...  ...       ...       ...  ..   
12     4279     172   1               716.523071  261  0.889747  0.979087   0   
       4444     45    0               763.679688  151  0.777646  0.974933   0   
       4502     127   19             -763.117737  118  0.897586  0.999999   0   
       6150     236   17             -627.641968   83  0.888946  0.985770   0   
       6645     120   0              -558.081665  200  0.852075  0.965248   0   

                                                             ...  \
                                     producer          nuid  ...   
                                              crlongtrkdiry  ...   
                                                             ...   
                                                             ...   
                                                             ...   
sample __ntuple entry rec.slc..index                         ...   
0      10       6     0                     0     -0.117672  ...   
       49       2     1                     0     -0.187669  ...   
       87       20    10                    1      0.068722  ...   
       105      15    14                    1     -0.114128  ...   
       110      13    0                     0      0.153792  ...   
..                                        ...           ...  ...   
12     4279     172   1                     0     -0.025453  ...   
       4444     45    0                     0      0.026420  ...   
       4502     127   19                    1     -0.062157  ...   
       6150     236   17                    1     -0.068047  

For the saved dataframes below...
- each row maps onto an ALP mass
- each column maps onto a tested fa value (refer back to unc_sensitivity.ipynb to see which ones)
- the actual entries in the dataframe are the total number of selected events at that mass and fa (and the cl specified by the file name). 

At present, I do not have per-event rescalings for the different parameter values saved.

Do that now.

In [4]:
# skip cell - not needed

new_ns_bycl = [
    'alp_new_ns_df_clDenom36',
    'alp_new_ns_df_clDenom100',
    'alp_new_ns_df_clDenom300',
    'alp_new_ns_df_clDenom1000'
]

df = pd.read_pickle(dir+'alp_new_ns_df_clDenom1000')
df

In [4]:
%%time

# idea for handling the list of lists issue:
# - create dictionary to map from c to the list I want

new_MS220_weights = []
new_MS240_weights = []
new_MS260_weights = []

new_MA300_weights = []
new_MA350_weights = []
new_MA400_weights = []
new_MA450_weights = []

for c in range(len(categories)-2):
    df = evtdf[categories[c]]
    print(categories[c].name)
    HPS = True # alternative is ALPs (make sure not to include nus and cosmics in this loop)
    if c>2: HPS = False 
    if HPS:
        old_th = old_thetas[c]
        new_mixing = best_thetas[c]
        print('Old, new theta: %a %a \n' %(old_th, new_mixing))
        mcdf = higgs_mcdfs[c]
    else:
        old_fa = old_fas[c-3]
        new_fa = best_fas[c-3]
        old_cl = old_cls[c-3]
        new_cl = best_cls[c-3]
        print('Old, new fa: %a %a' %(old_fa, new_fa))
        print('Old, new cl: %a %a \n' %(old_cl, new_cl))
        mcdf = alp_nosup_mcdfs[c-3]
    for idx in df.index:
        row = mcdf.loc[(idx[1],idx[2], 0)]
        if c == 0:
            new_MS220_weights.append(
                float(reweight_mixing(new_mixing, row.start, row.enter, row.exit, row.decay_length, old_th))
            )
        if c == 1:
            new_MS240_weights.append(
                float(reweight_mixing(new_mixing, row.start, row.enter, row.exit, row.decay_length, old_th))
            )
        if c == 2:
            new_MS260_weights.append(
                float(reweight_mixing(new_mixing, row.start, row.enter, row.exit, row.decay_length, old_th))
            )
        if c == 3:
            new_MA300_weights.append(
                float(reweight_alps(old_fa, new_fa, old_cl, new_cl, row.start, row.enter, 
                                    row.exit, row.decay_length, row.allowed_decay_fraction))
            )
        if c == 4:
            new_MA350_weights.append(
                float(reweight_alps(old_fa, new_fa, old_cl, new_cl, row.start, row.enter, 
                                    row.exit, row.decay_length, row.allowed_decay_fraction))
            )
        if c == 5:
            new_MA400_weights.append(
                float(reweight_alps(old_fa, new_fa, old_cl, new_cl, row.start, row.enter, 
                                    row.exit, row.decay_length, row.allowed_decay_fraction))
            )
        if c == 6:
            new_MA450_weights.append(
                float(reweight_alps(old_fa, new_fa, old_cl, new_cl, row.start, row.enter, 
                                    row.exit, row.decay_length, row.allowed_decay_fraction))
            )
            
weights = [
    new_MS220_weights,
    new_MS240_weights,
    new_MS260_weights,
    new_MA300_weights,
    new_MA350_weights,
    new_MA400_weights,
    new_MA450_weights
]

$M_S$ = 220, $\theta_S$ = 5e-05
Old, new theta: 5e-05 0.00095 

$M_S$ = 240, $\theta_S$ = 5e-05
Old, new theta: 5e-05 0.0007 

$M_S$ = 260, $\theta_S$ = 5e-05
Old, new theta: 5e-05 0.0001 

$M_{ALP}$ = 300, $c$ = 0.01, $fa$ = 100000.0
Old, new fa: 100000.0 0.0009
Old, new cl: 0.01 0.001 

$M_{ALP}$ = 350, $c$ = 0.01, $fa$ = 100000.0
Old, new fa: 100000.0 0.0009
Old, new cl: 0.01 0.001 

$M_{ALP}$ = 400, $c$ = 0.01, $fa$ = 100000.0
Old, new fa: 100000.0 0.0009
Old, new cl: 0.01 0.001 

$M_{ALP}$ = 450, $c$ = 0.01, $fa$ = 100000.0
Old, new fa: 100000.0 7e-05
Old, new cl: 0.01 0.0033333333333333335 

CPU times: user 1min 14s, sys: 1.02 s, total: 1min 15s
Wall time: 1min 13s


In [6]:
print(len(weights), '\n')
for w in weights:
    print(len(w))

7 

37
170
182
2872
2882
3668
3878


### Leaving off Monday 10/23:

Question:
I just generated the weights for just the most sensitive benchmark models. But to actually get kinematic distributions properly re-weighted for the rescaled model parameters, I would need to get the weights for the entire preselected sample, not just the events that survived the selection cuts. I think this will take WAY too long, given the selected ALP samples taking ~Order(30 minutes).

Worth it?
Not worth it?
Something we can do instead?

~

Actually, I think what I have just done is still useful: 

I can still see where in the distribution the selected events fall relative to the full preselected sample at nominal model benchmarks. 
I suppose I just won't be able to see how that compares to the signal events that were ELIMINATED.

Consider: Is there any reason to expect different shapes in the key distributions for BSM signals after rescaling model parameters?

In [7]:
# Copied from event selection nb, edited to include newly rescaled models:

def plot_scalar_numi_angle(df, bins = 25, vline=None, bsm=False,
                           density=True, POTSTR=POTSTR, sample_description=""):
    fig = plt.figure()
    ax = plt.subplot(111)
    
    var = df.Snumi_angle_mcs
    scale = df.scale.copy()
    var[np.isnan(var) | (var < 0)] = -10
    
    if bsm == True:
        categories = make_categories(df, bsm=True)
    else:
        categories = make_categories(df)
    for c in categories:
        _ = plt.hist(var[c], weights=scale[c], bins=bins, label=c.name, color=c.color,
                     histtype="step", linewidth=2, density=density, range=(0,0.5))
    
    #plt.legend()
    plt.title("ICARUS MC"+sample_description+" \n direction determined w/ MCS for $p_\\mu$")
    plt.xlabel("Parent Angle to Beam (rad)")
    if density:
        plt.ylabel("Area Normalized")
    else:
        plt.ylabel("Candidates / %s" % POTSTR)
    plt.yscale("log")
    if vline is not None:
        plt.axvline([vline], linestyle=":", color="r", linewidth=2)
    #plt.axvline([NuMI_angle_thresh], linestyle=":", color="r", linewidth=2)
    #plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)
    plt.tight_layout()
    
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc = 'center left', bbox_to_anchor=(1, 0.5))

    if dosave: plt.savefig(savedir + "max_shw_len_M%i.pdf" % HMASS)
    plt.show()
    


In [ ]:
plot_scalar_numi_angle(evtdf, vline=NuMI_angle_thresh, 
                       sample_description="\n [all that pass preselection]", density=False)


#nm1_cut_list = [not_stub_05, not_stub_1, not_stub_2, not_stub_3, # 'nm1' for 'n minus 1'
#                max_shw_len_mask, ok_chi2mu, ok_chi2p, 
#                open_angle_mask, shortTrk_len_mask]
#mask = apply_cuts(evtdf, nm1_cut_list)[-1]

#plot_scalar_numi_angle(evtdf[mask], vline=NuMI_angle_thresh, 
#                       sample_description="\n [n-1 cuts]", density=False)


In [ ]:
# scratch

new_ns_bycl = [
    'alp_new_ns_df_clDenom36',
    'alp_new_ns_df_clDenom100',
    'alp_new_ns_df_clDenom300',
    'alp_new_ns_df_clDenom1000'
]

# 7 is the number of benchmark masses we are testing.
blank = []
best_fas = [blank.copy()] * 7
peak_rates = [blank.copy()] * 7
              
# ISSUE: appending lists within lists affects all lists in the superlist.

m300_best_fa = [] # will have same length as number of cl_s I tested.
m300_peak_rate = []

for n in range(len(new_ns_bycl)):
    df = pd.read_pickle(dir+new_ns_bycl[n])